# Bank Marketing Conversion Prediction

## Introduction

The datasets used in this report are from direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. The aim of the phone calls, often consisting of several contacts to the same client, is to access the clients' willingness to subscribe the bank product.


The goal is to predict if the client will subscribe a term deposit.

## Dataset Description

Dataset source: 

S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. In P. Novais et al. (Eds.), Proceedings of the European Simulation and Modelling Conference - ESM'2011, pp. 117-121, Guimarães, Portugal, October, 2011. EUROSIS.

There are two datasets: 1) bank-full.csv with all examples, time-ordered (from May 2008 to November 2010). 2) bank.csv with 10% of the examples (4521), randomly selected from bank-full.csv. The smallest dataset is provided to test more computationally demanding machine learning algorithms (e.g. SVM).

## 

In [25]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

## Data Exploration

In [19]:
## Load bank-full.csv

data_bank_full = pd.read_csv("bank-full.csv",sep=';')

In [18]:
data_bank_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
age          45211 non-null int64
job          45211 non-null object
marital      45211 non-null object
education    45211 non-null object
default      45211 non-null object
balance      45211 non-null int64
housing      45211 non-null object
loan         45211 non-null object
contact      45211 non-null object
day          45211 non-null int64
month        45211 non-null object
duration     45211 non-null int64
campaign     45211 non-null int64
pdays        45211 non-null int64
previous     45211 non-null int64
poutcome     45211 non-null object
y            45211 non-null object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [21]:
data_bank_full.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


### Explore the "Job" column

In [17]:
data_bank_full["job"].describe()

count           45211
unique             12
top       blue-collar
freq             9732
Name: job, dtype: object

In [22]:
different_jobs = set()
for i in data_bank_full.index:
    different_jobs.add(data_bank_full.iloc[i]['job'])

print(different_jobs)

{'management', 'unemployed', 'blue-collar', 'services', 'student', 'unknown', 'housemaid', 'technician', 'self-employed', 'entrepreneur', 'retired', 'admin.'}


### Convert categorical variables into dummy variables with pandas.get_dummies

In [26]:
cat_features = [ 'job', 'marital', 'education', 'default', 'housing',
       'loan', 'contact', 'month','poutcome']
data_bank_full = pd.get_dummies(data_bank_full, columns=cat_features)

In [28]:
data_bank_full['y'] = (data_bank_full['y']=='yes').astype(int)

In [29]:
data_bank_full.head(10)

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,2143,5,261,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,44,29,5,151,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,33,2,5,76,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,47,1506,5,92,1,-1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
4,33,1,5,198,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
5,35,231,5,139,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
6,28,447,5,217,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
7,42,2,5,380,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
8,58,121,5,50,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
9,43,593,5,55,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [38]:
## number of 'y = yes' / number of 'y = no'

(data_bank_full[data_bank_full['y']==1]['y'].count())/(data_bank_full[data_bank_full['y']==0]['y'].count())

0.1324833425179099

## Build Predictive Models

### Logistic Regression

In [39]:
from sklearn.model_selection import train_test_split